In [1]:
import os
import librosa
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
AUDIO_ROOT = r"C:\Users\Rishi S Etagi\Desktop\extracted_audio\combined audio"
LABELS_CSV_PATH = r"C:\Users\Rishi S Etagi\Desktop\medivoice\combined_data_disease.csv"
OUTPUT_CSV_PATH = "final_dataset.csv"

In [3]:
def extract_features_from_audio(file_path):
    try:
        y, sr = librosa.load(file_path, sr=None)

        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        mfcc_mean = np.mean(mfcc, axis=1)

        mel = librosa.feature.melspectrogram(y=y, sr=sr)
        mel_mean = np.mean(mel, axis=1)[:13]  

        pitch, _ = librosa.piptrack(y=y, sr=sr)
        pitch_mean = np.mean(pitch[pitch > 0]) if np.any(pitch > 0) else 0

        zcr = np.mean(librosa.feature.zero_crossing_rate(y))
        spec_centroid = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
        spec_bw = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
        rolloff = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))
        rms = np.mean(librosa.feature.rms(y=y))

        features = np.concatenate([
            mfcc_mean,
            mel_mean,
            [pitch_mean, zcr, spec_centroid, spec_bw, rolloff, rms]
        ])

        return features

    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None


In [4]:
def extract_features_per_folder(data_path):
    all_features = []
    folder_names = []

    for folder in tqdm(os.listdir(data_path)):
        folder_path = os.path.join(data_path, folder)
        if not os.path.isdir(folder_path):
            continue

        folder_features = []

        for file in os.listdir(folder_path):
            if file.endswith(".wav"):
                file_path = os.path.join(folder_path, file)
                features = extract_features_from_audio(file_path)
                if features is not None:
                    folder_features.append(features)

        if folder_features:
            folder_features = np.mean(folder_features, axis=0)  
            all_features.append(folder_features)
            folder_names.append(folder)

    features_df = pd.DataFrame(all_features)
    features_df.insert(0, 'participant_id', folder_names)

    return features_df


In [5]:
def preprocess_labels(labels_df):
    labels_df.columns = labels_df.columns.str.lower().str.strip()

    labels_df = labels_df.rename(columns={'a': 'age', 'g': 'gender'})

    required_columns = ['id', 'age', 'gender', 'diabetes', 'ht', 'asthma', 'fever', 'smoker', 'cld', 'ihd']
    labels_df = labels_df[required_columns]

    labels_df['id'] = labels_df['id'].astype(str)
    labels_df['gender'] = labels_df['gender'].map({'male': 0, 'female': 1})

    bool_fields = ['diabetes', 'ht', 'asthma', 'fever', 'smoker', 'cld', 'ihd']
    for col in bool_fields:
        labels_df[col] = labels_df[col].fillna('').apply(lambda x: 1 if str(x).strip().lower() == 'true' else 0)

    return labels_df


In [6]:
if __name__ == "__main__":
    print("Extracting features.")
    features_df = extract_features_per_folder(AUDIO_ROOT)

    print("Loading labels.")
    labels_df = pd.read_csv(LABELS_CSV_PATH)
    labels_df = preprocess_labels(labels_df)

    print("Merging features with labels.")
    features_df['participant_id'] = features_df['participant_id'].astype(str)
    merged_df = pd.merge(features_df, labels_df, left_on='participant_id', right_on='id')
    merged_df.drop(columns=['id'], inplace=True)

    print("Saving final dataset to:", OUTPUT_CSV_PATH)
    merged_df.to_csv(OUTPUT_CSV_PATH, index=False)

    print("Done", merged_df.shape)

Extracting features.


 12%|█▏        | 54/434 [01:56<11:36,  1.83s/it] c:\Users\Rishi S Etagi\Desktop\medivoice\.venv\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(


Error processing C:\Users\Rishi S Etagi\Desktop\extracted_audio\combined audio\697xpWya0DbSEN4Y6tsEH3BchHw1\breathing-deep.wav: can't extend empty axis 0 using modes other than 'constant' or 'empty'
Error processing C:\Users\Rishi S Etagi\Desktop\extracted_audio\combined audio\697xpWya0DbSEN4Y6tsEH3BchHw1\breathing-shallow.wav: can't extend empty axis 0 using modes other than 'constant' or 'empty'
Error processing C:\Users\Rishi S Etagi\Desktop\extracted_audio\combined audio\697xpWya0DbSEN4Y6tsEH3BchHw1\cough-heavy.wav: can't extend empty axis 0 using modes other than 'constant' or 'empty'
Error processing C:\Users\Rishi S Etagi\Desktop\extracted_audio\combined audio\697xpWya0DbSEN4Y6tsEH3BchHw1\cough-shallow.wav: can't extend empty axis 0 using modes other than 'constant' or 'empty'
Error processing C:\Users\Rishi S Etagi\Desktop\extracted_audio\combined audio\697xpWya0DbSEN4Y6tsEH3BchHw1\counting-fast.wav: can't extend empty axis 0 using modes other than 'constant' or 'empty'
Error p

 21%|██        | 89/434 [03:02<11:10,  1.94s/it]

Error processing C:\Users\Rishi S Etagi\Desktop\extracted_audio\combined audio\9Rhe6RDHuMNXaGiqKVaspw71Exo1\breathing-deep.wav: can't extend empty axis 0 using modes other than 'constant' or 'empty'
Error processing C:\Users\Rishi S Etagi\Desktop\extracted_audio\combined audio\9Rhe6RDHuMNXaGiqKVaspw71Exo1\breathing-shallow.wav: can't extend empty axis 0 using modes other than 'constant' or 'empty'
Error processing C:\Users\Rishi S Etagi\Desktop\extracted_audio\combined audio\9Rhe6RDHuMNXaGiqKVaspw71Exo1\cough-heavy.wav: can't extend empty axis 0 using modes other than 'constant' or 'empty'
Error processing C:\Users\Rishi S Etagi\Desktop\extracted_audio\combined audio\9Rhe6RDHuMNXaGiqKVaspw71Exo1\cough-shallow.wav: can't extend empty axis 0 using modes other than 'constant' or 'empty'
Error processing C:\Users\Rishi S Etagi\Desktop\extracted_audio\combined audio\9Rhe6RDHuMNXaGiqKVaspw71Exo1\counting-fast.wav: can't extend empty axis 0 using modes other than 'constant' or 'empty'
Error p

...

Error processing C:\Users\Rishi S Etagi\Desktop\extracted_audio\combined audio\VcmjCfW6ddgE9lFhDXTO6hDlb452\counting-fast.wav: can't extend empty axis 0 using modes other than 'constant' or 'empty'


 87%|████████▋ | 378/434 [12:19<02:15,  2.42s/it]

Error processing C:\Users\Rishi S Etagi\Desktop\extracted_audio\combined audio\Vq1h51z5x3Wp4wS2pCm6yEAZvu82\breathing-deep.wav: can't extend empty axis 0 using modes other than 'constant' or 'empty'
Error processing C:\Users\Rishi S Etagi\Desktop\extracted_audio\combined audio\Vq1h51z5x3Wp4wS2pCm6yEAZvu82\breathing-shallow.wav: can't extend empty axis 0 using modes other than 'constant' or 'empty'
Error processing C:\Users\Rishi S Etagi\Desktop\extracted_audio\combined audio\Vq1h51z5x3Wp4wS2pCm6yEAZvu82\cough-heavy.wav: can't extend empty axis 0 using modes other than 'constant' or 'empty'
Error processing C:\Users\Rishi S Etagi\Desktop\extracted_audio\combined audio\Vq1h51z5x3Wp4wS2pCm6yEAZvu82\cough-shallow.wav: can't extend empty axis 0 using modes other than 'constant' or 'empty'
Error processing C:\Users\Rishi S Etagi\Desktop\extracted_audio\combined audio\Vq1h51z5x3Wp4wS2pCm6yEAZvu82\counting-fast.wav: can't extend empty axis 0 using modes other than 'constant' or 'empty'
Error p

100%|██████████| 434/434 [14:07<00:00,  1.95s/it]

Loading labels.
Merging features with labels.
Saving final dataset to: final_dataset.csv
Done (425, 42)
